In [41]:
import os
import copy
import numpy as np
import os
import json
import glob
import random


In [42]:
# file_path = '/home/shahpouriz/Data/DBP_CTs/DBP_OP042_coordination_data.json'

In [43]:
import glob
import os
import json

class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        # Search for json files matching the pattern
        file_pattern = os.path.join(self.directory_path, '*_coordination_data.json')
        json_files = glob.glob(file_pattern)

        results = []
        for file_path in json_files:
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Initialize a dictionary to store the extracted data for this file
            file_results = {
                'id': data.get('id'),
                'examination_details': []
            }

            # Iterate through each examination detail
            for examination in data.get('examination_details', []):
                # Filter for 'A1PHH' plans
                for plan in examination.get('plans', []):
                    # if plan.get('plan_name') == 'A1PHH' and plan.get('planning_exam_name') == 'pCTp0':
                        # Extract and add the relevant plan details
                        file_results['examination_details'].append({
                            'plan_name': plan.get('plan_name'),
                            'planning_exam_name': plan.get('planning_exam_name'),
                            'repeatedCT_name': plan.get('repeatedCT_name'),
                            'final_translation_coordinate': plan.get('final_translation_coordinate'),
                            'planningCT_filename': plan.get('planningCT_filename'),
                            'repeatedCT_filename': plan.get('repeatedCT_filename')
                        })
            
            # Add the file's results to the overall results list
            results.append(file_results)

        return results

# Usage
directory_path = '/home/shahpouriz/Data/DBP_CTs'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

# # Optionally, print the results to verify
# for result in all_results:
#     print(result)
all_results

[{'id': 'DBP_OP055',
  'examination_details': [{'plan_name': 'A1PHH',
    'planning_exam_name': 'pCTp0',
    'repeatedCT_name': 'rCTp1',
    'final_translation_coordinate': {'x': -10.937396994141649,
     'y': -1.9845064788287614,
     'z': -0.2840079647225169},
    'planningCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/pCTp0',
    'repeatedCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/rCTp1'},
   {'plan_name': 'A1PHH',
    'planning_exam_name': 'pCTp0',
    'repeatedCT_name': 'rCTp4',
    'final_translation_coordinate': {'x': 7.519069220000007,
     'y': 3.1272398513592425,
     'z': -10.168307720214337},
    'planningCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/pCTp0',
    'repeatedCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/rCTp4'},
   {'plan_name': 'A1PHH',
    'planning_exam_name': 'pCTp0',
    'repeatedCT_name': 'rCTp9',
    'final_translation_coordinate': {'x': -2.3478576158662112,
     'y': 6.621074728213304,
     '

In [44]:
import os
import SimpleITK as sitk

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory_proton, output_directory_foton):
        self.directory_path = directory_path
        self.output_directory_proton = output_directory_proton
        self.output_directory_foton = output_directory_foton
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()
        for patient in patient_data:
            patient_id = patient['id']
            # print(patient_id)
            for plan_index, plan in enumerate(patient['examination_details']):
                # Determine the output path based on whether the plan is for proton or foton
                output_path = self.output_directory_proton if 'P' in plan['plan_name'] else self.output_directory_foton
                
                full_output_path = os.path.join(output_path, patient_id)
                if not os.path.exists(full_output_path):
                    os.makedirs(full_output_path)
                # Process files for this plan
                plan_name = plan.get('planning_exam_name')
            
                nrrd_path = self._process_dose_file(plan['planningCT_filename'], full_output_path, patient_id, plan_name, "planningCT")
                plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']
                print(plan)
                # for exam_index, exam in enumerate(plan.get('evaluation_examinations', [])):
                #     print(exam)
                exam_index = plan.get('repeatedCT_name')
                nrrd_path = self._process_dose_file(plan['repeatedCT_filename'], full_output_path, patient_id, plan_name, "repeatedCT", exam_index)
                plan['repeatedCT_filename'] = nrrd_path or plan['repeatedCT_filename']
     # Optional: Return the updated patient data for verification
        return patient_data
    

    def _process_dose_file(self, dose_dicom_dir, output_path, patient_id, plan_name, exam_type, exam_index=None):
        
        if dose_dicom_dir is None:
            # print(f"Error: The dose_dicom_dir is None for patient_id: {patient_id}, plan_name: {plan_name}")
            return None
        # Generate the full path to the DICOM directory
        full_dose_path = os.path.join(self.directory_path, dose_dicom_dir) if self.directory_path not in dose_dicom_dir else dose_dicom_dir
        
        # Check if the directory exists
        if not os.path.isdir(full_dose_path):
            # print(f"Directory not found: {full_dose_path}")
            return None
            
        # Construct NRRD filename
        nrrd_filename = f"{patient_id}_{plan_name}_{exam_type}{'' if exam_index is None else '_' + exam_index}.nrrd"
        nrrd_path = os.path.join(output_path, nrrd_filename)
        
        # Debugging print
        # print(f"Processing DICOM series from: {full_dose_path}")
        
        # Attempt to read the DICOM series and write as NRRD
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(full_dose_path)
        if not dicom_names:  # Check if the DICOM names list is empty
            # print(f"No DICOM files found in directory: {full_dose_path}")
            return None
        
        reader.SetFileNames(dicom_names)
        try:
            image = reader.Execute()
            sitk.WriteImage(image, nrrd_path)
            print(f"Saved NRRD file at {nrrd_path}")
            return nrrd_filename
        except RuntimeError as e:
            print(f"Error reading DICOM series from {full_dose_path}: {e}")
            return None


In [45]:
output_directory_proton = '/data/shahpouriz/DBP_CTs/nrrd/proton'
output_directory_foton = '/data/shahpouriz/DBP_CTs/nrrd/foton'

converter = DICOMtoNRRDConverter(directory_path, output_directory_proton, output_directory_foton)
updated_patient_data = converter.convert_and_save()

# Print the updated patient data
# for patient in updated_patient_data:
#     print(patient)

Saved NRRD file at /data/shahpouriz/DBP_CTs/nrrd/proton/DBP_OP055/DBP_OP055_pCTp0_planningCT.nrrd
{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp1', 'final_translation_coordinate': {'x': -10.937396994141649, 'y': -1.9845064788287614, 'z': -0.2840079647225169}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/rCTp1'}
Saved NRRD file at /data/shahpouriz/DBP_CTs/nrrd/proton/DBP_OP055/DBP_OP055_pCTp0_repeatedCT_rCTp1.nrrd
Saved NRRD file at /data/shahpouriz/DBP_CTs/nrrd/proton/DBP_OP055/DBP_OP055_pCTp0_planningCT.nrrd
{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp4', 'final_translation_coordinate': {'x': 7.519069220000007, 'y': 3.1272398513592425, 'z': -10.168307720214337}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': '/home/shahpouriz/Data/DBP_CTs/DBP_OP055/A1PHH/rCTp4'}
Saved NRRD file at /data/shahpouriz/DBP_CTs

In [46]:
# Print the updated patient data
for patient in updated_patient_data:
    print(patient)


{'id': 'DBP_OP055', 'examination_details': [{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp1', 'final_translation_coordinate': {'x': -10.937396994141649, 'y': -1.9845064788287614, 'z': -0.2840079647225169}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': 'DBP_OP055_pCTp0_repeatedCT_rCTp1.nrrd'}, {'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp4', 'final_translation_coordinate': {'x': 7.519069220000007, 'y': 3.1272398513592425, 'z': -10.168307720214337}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': 'DBP_OP055_pCTp0_repeatedCT_rCTp4.nrrd'}, {'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp9', 'final_translation_coordinate': {'x': -2.3478576158662112, 'y': 6.621074728213304, 'z': -14.367127383584787}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': 'DBP_OP055_pCTp0_repeatedCT_rCTp9.nrrd'}, {'plan_nam

In [47]:
initial_count = len(updated_patient_data)
print(f"Initial patient count: {initial_count}")

filtered_patient_data = []

for patient in updated_patient_data:
    keep_patient = True
    if 'examination_details' not in patient:
        print(f"Skipping patient {patient.get('id', 'unknown')} due to missing examination_details.")
        continue  # Skip patients without examination_details entirely
    for plan in patient['examination_details']:  # Directly iterate through what we previously considered as 'plans'
        if plan.get('planningCT_filename') is None or plan.get('repeatedCT_filename') is None:
            print(f"Excluding plan due to missing filenames in patient {patient.get('id', 'unknown')}.")
            keep_patient = False
            break
    if keep_patient:
        filtered_patient_data.append(patient)

print(f"Filtered patient count: {len(filtered_patient_data)}")

# Optionally, print details of the filtered patients to verify the output
for patient in filtered_patient_data:
    print(patient)


Initial patient count: 56
Excluding plan due to missing filenames in patient DBP_OP038.
Excluding plan due to missing filenames in patient DBP_OP068.
Excluding plan due to missing filenames in patient DBP_OP021.
Excluding plan due to missing filenames in patient DBP_OP037.
Excluding plan due to missing filenames in patient DBP_OP003.
Excluding plan due to missing filenames in patient DBP_OP036.
Excluding plan due to missing filenames in patient DBP_OP006.
Excluding plan due to missing filenames in patient DBP_OP070.
Excluding plan due to missing filenames in patient DBP_OP069.
Excluding plan due to missing filenames in patient DBP_OP005.
Filtered patient count: 46
{'id': 'DBP_OP055', 'examination_details': [{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'repeatedCT_name': 'rCTp1', 'final_translation_coordinate': {'x': -10.937396994141649, 'y': -1.9845064788287614, 'z': -0.2840079647225169}, 'planningCT_filename': 'DBP_OP055_pCTp0_planningCT.nrrd', 'repeatedCT_filename': 'DBP_OP0

In [48]:
# Save updated_patient_data to a JSON file
output_json_file = output_directory_proton + '/file_info.json' 
with open(output_json_file, 'w') as file:
    json.dump(filtered_patient_data, file, indent=4)
    
print(f"Updated patient data saved to {output_json_file}")

Updated patient data saved to /data/shahpouriz/DBP_CTs/nrrd/proton/file_info.json
